In [9]:
import cv2
import numpy as np

# File paths
config_file = "C:\\Users\\vishn\\OneDrive\\Desktop\\mini_project\\config.pbtxt"
frozen_model = "C:\\Users\\vishn\\OneDrive\\Desktop\\mini_project\\frozen_inference_graph.pb"
labels_file = "C:\\Users\\vishn\\OneDrive\\Desktop\\mini_project\\labels2.names"

# Load class labels
classLabels = []

with open(labels_file, 'rt') as f:
    classLabels = f.read().rstrip('\n').split('\n')

# Load the detection model
model = cv2.dnn_DetectionModel(frozen_model, config_file)
model.setInputSize(320, 320)  # Model input size
model.setInputScale(1.0 / 127.5)  # Normalize image pixel values
model.setInputMean((127.5, 127.5, 127.5))  # Subtract mean
model.setInputSwapRB(True)  # Swap Red and Blue channels if required

# Open video source
cap = cv2.VideoCapture("C:\\Users\\vishn\\OneDrive\\Desktop\\mini_project\\video\\cctv.mp4")
if not cap.isOpened():
    cap = cv2.VideoCapture(0)  # Fallback to webcam if video cannot be loaded
if not cap.isOpened():
    raise IOError("Cannot open video source")

# Initialize font parameters
font_scale = 1
font = cv2.FONT_HERSHEY_SIMPLEX

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    # Object detection
    try:
        ClassIndex, confidence, bbox = model.detect(frame, confThreshold=0.55)
    except Exception as e:
        print(f"Detection error: {e}")
        continue

    # Draw bounding boxes and labels
    if len(ClassIndex) != 0:
        for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
            if ClassInd <= len(classLabels):  # Ensure index is within range
                x, y, w, h = boxes
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(frame, f"{classLabels[ClassInd - 1]}: {conf:.2f}", (x, y - 10),
                            font, font_scale, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Object Detection", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
